In [1]:
import tensorflow as tf
import pandas as pd
import re
train = pd.read_csv("./training_data.csv", header=0, delimiter="\t|\n")
data = train["comment"]
### Text Preprocessing
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
keep_stopwords = stopwords.words("english")[-36:]+stopwords.words("english")[131:133] + stopwords.words("english")[116:120]
new_stopwords = set(stopwords.words("english")).difference(set(keep_stopwords))
replace_num = "[-+]?[0-9]*\.?[0-9]+"
replace_url = "http(s?)://[^\s]+"
def review_to_words(raw_review):
    
    review_text = BeautifulSoup(raw_review).get_text()
    review_text = re.sub(r'[\t\r\n]'," ",review_text)
    url = re.sub(replace_url,"URL",raw_review)
    num = re.sub(replace_num,"NUM",url)
    letters_only = re.sub("[^a-zA-Z']", " ", num) #Also keep single quote
    
    words = letters_only.lower().split()
    
    meaningful_words = [w for w in words if not w in new_stopwords]
    
    return(" ".join(meaningful_words))

clean_data = dict()    
clean_data["x"] = []
clean_data["y"] = []
import bar
step=0
total = len(data)
for each in data:
    clean_data["x"].append(review_to_words(each))
    bar.drawProgressBar(step/total)
    step +=1


clean_data["y"] = train["score"]

/home/kevindong1994/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/home/kevindong1994/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  after removing the cwd from sys.path.


[                      ] 0.11%

/home/kevindong1994/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /home/kevindong1994/anaconda3/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


[ =============        ] 65.90%

/home/kevindong1994/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:219: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)


[ ===================  ] 100.00%

In [2]:
import numpy as np
def int2vector(i):
    return np.array([1]*(i+1)+[0]*(10-i)).reshape(11,1)
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(clean_data["x"], clean_data["y"], test_size=0.33, random_state=42)

clean_data["y"] = [int2vector(i) for i in clean_data["y"]]

In [3]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(clean_data["x"], clean_data["y"], test_size=0.33, random_state=42)

In [11]:
from sklearn.feature_extraction.text import CountVectorizer  
num_features = 5000
vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = num_features) 
train_data_features = vectorizer.fit_transform(X_train)
train_data_features = train_data_features.toarray()


In [12]:
train_data = [train_data_features[i,] for i in range(train_data_features.shape[0])]

In [130]:
batch_size = 100
epochs = 50
epoch = 0
ids = list(range(0,train_data_features.shape[0],batch_size))
i=0
def generate_batch(features,labels):
    global i
    global ids
    global epoch
    if i==len(ids)-1:
        inner_i = i
        i = 0
        epoch += 1
        return features[ids[inner_i]:],labels[ids[inner_i]:]
    else:
        inner_i = i
        i +=1
        return features[ids[inner_i]:ids[i]],labels[ids[inner_i]:ids[i]]
    


In [131]:
len(train_data)

101722

In [132]:
H = 20
graph = tf.Graph()
with graph.as_default():
    x = tf.placeholder(tf.float32,shape=[None,num_features])
    y_ = tf.placeholder(tf.float32,shape=[None,11,1])
    new_y = tf.squeeze(y_)
    W_1 = tf.Variable(tf.random_uniform([num_features,H]))
    b_1 = tf.Variable(tf.zeros([H]))
    
    W_2 = tf.Variable(tf.random_uniform([H,11]))
    b_2 = tf.Variable(tf.zeros([11]))
    
    temp_1 = tf.nn.relu(tf.matmul(x,W_1) + b_1)
    temp_2 = tf.matmul(temp_1,W_2) + b_2
    output = tf.nn.sigmoid(temp_2)
    #loss = tf.reduce_mean(tf.square(tf.subtract(output,new_y)))
    loss = tf.losses.mean_squared_error(output,new_y)
    opt = tf.train.AdamOptimizer(learning_rate=0.01)
    opt_op = opt.minimize(loss)
    init = tf.global_variables_initializer()
    
# with tf.Session(graph = graph) as session:

sess = tf.InteractiveSession(graph=graph)
sess.run(init)
print("Initialized")

Initialized


In [133]:
step = 0 
while epoch<epochs:
    step+=1
    batch_x,batch_y = generate_batch(train_data,y_train)
    feed_dict={x:batch_x,y_:batch_y}
    _,loss_val = sess.run([opt_op,loss],feed_dict=feed_dict)
    if step % 2000 == 0:
        print("Step: %i, Epoch: %i, Loss:%f"%(step,epoch,loss_val))
    

Step: 2000, Epoch: 1, Loss:0.084446
Step: 4000, Epoch: 3, Loss:0.082096
Step: 6000, Epoch: 5, Loss:0.058904
Step: 8000, Epoch: 7, Loss:0.058330
Step: 10000, Epoch: 9, Loss:0.062186
Step: 12000, Epoch: 11, Loss:0.052014
Step: 14000, Epoch: 13, Loss:0.061606
Step: 16000, Epoch: 15, Loss:0.054956
Step: 18000, Epoch: 17, Loss:0.059480
Step: 20000, Epoch: 19, Loss:0.057726
Step: 22000, Epoch: 21, Loss:0.072094
Step: 24000, Epoch: 23, Loss:0.049781
Step: 26000, Epoch: 25, Loss:0.048849
Step: 28000, Epoch: 27, Loss:0.047707
Step: 30000, Epoch: 29, Loss:0.058079
Step: 32000, Epoch: 31, Loss:0.051351
Step: 34000, Epoch: 33, Loss:0.040188
Step: 36000, Epoch: 35, Loss:0.032521
Step: 38000, Epoch: 37, Loss:0.055262
Step: 40000, Epoch: 39, Loss:0.058417
Step: 42000, Epoch: 41, Loss:0.044782
Step: 44000, Epoch: 43, Loss:0.052051
Step: 46000, Epoch: 45, Loss:0.055085
Step: 48000, Epoch: 47, Loss:0.059773
Step: 50000, Epoch: 49, Loss:0.050142


In [47]:
batch_x,batch_y = generate_batch(train_data,y_train)

In [48]:
loss.eval(feed_dict={x:batch_x,y_:batch_y})

0.16667761

In [94]:
def to_predict(row):
    for i in range(11):
        if row[-1]>0.5:
            return 10
        elif row[i]<0.5:
            return i-1
test_data_features = vectorizer.transform(X_test)
test_data_features = test_data_features.toarray()
test_data = [test_data_features[i,] for i in range(test_data_features.shape[0])]


In [137]:
result = sess.run(output,feed_dict={x:test_data_features})


In [138]:
result_label = [to_predict(each) for each in result]

In [77]:
_1, _2, _3, new_y_test = train_test_split(clean_data["x"], clean_data["y"], test_size=0.33, random_state=42)

In [139]:
from sklearn.metrics import mean_squared_error
mean_squared_error(result_label, new_y_test)

4.912298910223145

In [88]:
new_y_test = list(new_y_test)
print(new_y_test)

[10, 10, 1, 10, 10, 6, 8, 10, 10, 10, 5, 10, 5, 8, 10, 7, 10, 0, 8, 10, 10, 8, 10, 8, 8, 10, 10, 10, 10, 8, 10, 10, 10, 10, 10, 7, 8, 7, 7, 5, 7, 10, 10, 9, 9, 9, 8, 5, 10, 10, 10, 7, 10, 7, 7, 9, 10, 9, 9, 8, 10, 10, 7, 9, 9, 8, 8, 9, 10, 10, 10, 7, 5, 7, 10, 2, 10, 7, 10, 10, 6, 7, 7, 8, 7, 10, 5, 10, 8, 10, 7, 8, 8, 10, 10, 8, 8, 7, 10, 9, 10, 10, 10, 9, 7, 9, 10, 7, 9, 10, 8, 10, 5, 10, 6, 8, 9, 2, 7, 5, 8, 7, 10, 7, 5, 10, 10, 10, 8, 9, 10, 10, 10, 10, 3, 10, 9, 10, 9, 10, 8, 6, 8, 7, 10, 10, 8, 5, 10, 8, 7, 8, 7, 8, 8, 10, 7, 3, 8, 6, 4, 6, 10, 10, 7, 9, 8, 9, 5, 2, 10, 8, 6, 9, 9, 10, 7, 10, 10, 7, 7, 10, 10, 10, 6, 1, 8, 6, 7, 4, 8, 5, 10, 9, 10, 10, 7, 7, 6, 6, 1, 2, 7, 9, 9, 10, 9, 10, 6, 10, 8, 6, 4, 10, 6, 8, 7, 9, 7, 10, 9, 10, 10, 7, 7, 3, 10, 9, 10, 8, 6, 9, 3, 9, 9, 9, 10, 9, 10, 8, 8, 10, 10, 0, 9, 10, 10, 7, 5, 7, 10, 8, 7, 7, 8, 10, 10, 10, 5, 9, 10, 9, 10, 9, 10, 10, 10, 10, 10, 10, 9, 7, 9, 10, 8, 9, 10, 10, 8, 10, 10, 8, 0, 10, 10, 8, 10, 8, 7, 5, 6, 10, 10, 9, 10

In [96]:
s=0
for i in range(50102):
    s += (result_label[i]-new_y_test[i])^2

In [108]:
def pingfen(pred,real):
    s = 0
    for i in range(len(pred)):
        s += (pred[i]-real[i])^2
    return s/len(pred)

In [93]:
print(to_predict(result[1]))
print(result[1])

None
[1.         1.         1.         1.         1.         1.
 1.         0.99999905 0.999777   0.99829227 0.75950867]


In [97]:
s/len(result_label)

0.5217755778212446

In [101]:
sum(np.array(new_y_test)==np.array(result_label))

16431

In [102]:
len(result_label)

50102

In [103]:
16431/50102

0.32795098000079836

In [110]:
pingfen(result_label,new_y_test)

0.5217755778212446

In [112]:
[result_label[i]-new_y_test[i] for i in range(50102)]

[-1,
 0,
 6,
 -5,
 0,
 3,
 -2,
 -2,
 -2,
 -1,
 1,
 0,
 0,
 -1,
 -1,
 -4,
 0,
 2,
 1,
 0,
 -2,
 1,
 0,
 0,
 -2,
 -4,
 0,
 0,
 -2,
 -1,
 0,
 0,
 0,
 0,
 -1,
 0,
 -7,
 0,
 2,
 2,
 2,
 0,
 -1,
 -2,
 0,
 0,
 2,
 4,
 -5,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 -1,
 1,
 -1,
 0,
 0,
 1,
 1,
 1,
 0,
 -1,
 0,
 0,
 0,
 -1,
 3,
 0,
 1,
 -1,
 3,
 0,
 -1,
 0,
 0,
 2,
 -2,
 -2,
 1,
 2,
 -1,
 4,
 0,
 1,
 -1,
 2,
 2,
 -2,
 -2,
 -2,
 2,
 1,
 -1,
 -8,
 1,
 -1,
 0,
 0,
 -3,
 0,
 1,
 -3,
 2,
 0,
 -1,
 1,
 0,
 0,
 -1,
 4,
 1,
 -4,
 4,
 -3,
 -1,
 2,
 1,
 0,
 3,
 5,
 -2,
 -1,
 0,
 1,
 1,
 -1,
 -1,
 0,
 0,
 7,
 0,
 -1,
 0,
 0,
 0,
 2,
 4,
 0,
 2,
 -2,
 0,
 -1,
 3,
 -1,
 -3,
 2,
 1,
 1,
 -3,
 -5,
 -2,
 2,
 6,
 -1,
 2,
 4,
 0,
 -3,
 -1,
 1,
 -1,
 -8,
 1,
 0,
 6,
 0,
 2,
 0,
 -2,
 0,
 -3,
 3,
 -1,
 0,
 2,
 -3,
 0,
 -3,
 -4,
 0,
 8,
 -3,
 3,
 -3,
 1,
 -2,
 4,
 -3,
 -1,
 0,
 0,
 2,
 0,
 -3,
 1,
 9,
 8,
 0,
 1,
 0,
 0,
 -1,
 0,
 1,
 -1,
 -3,
 1,
 1,
 0,
 -4,
 2,
 2,
 -1,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 2,
 -10,
 0,
 0,
 0,
 0

In [115]:
4.84*50102

242493.68